In [1]:
import brainpy as bp
import brainpy.math as bm
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5,6"  # specify which GPU(s) to be used
bm.disable_gpu_memory_preallocation()
bm.set_platform('gpu')

/home/spirit/anaconda3/envs/slca/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ML_Scale(bp.NeuGroup):
  def __init__(self, size, method='rk2', **kwargs):
    super(ML_Scale, self).__init__(size=size, **kwargs)
    # parameters
    self.V_Ca = 120./5.
    self.g_Ca = 4.
    self.V_K = -84./5.
    self.g_K = 8.
    self.V_leak = -60./5.
    self.g_leak = 2.
    self.C = 2./5.
    self.V1 = -1.2
    self.V2 = 18
    self.V_th = -15./5.


    self.V3 = 12.
    self.V4 = 17.
    self.phi = 2/3

    # variables
    self.V = bm.Variable(-40*bm.ones(self.num))
    self.W = bm.Variable(bm.zeros(self.num))
    self.input = bm.Variable(bm.zeros(self.num))
    self.spike = bm.Variable(bm.zeros(self.num, dtype=bool))

    # functions
    def dV(V, t, W, input):
        M_inf = (1 / 2) * (1 + bm.tanh((V*5. - self.V1) / self.V2))
        I_Ca = self.g_Ca * M_inf * (V - self.V_Ca)
        I_K = self.g_K * W * (V - self.V_K)
        I_Leak = self.g_leak * (V - self.V_leak)
        dVdt = (- I_Ca - I_K - I_Leak + input*2.) / self.C
        return dVdt

    def dW(W, t, V):
        tau_W = 1 / (self.phi * 5. * bm.cosh((V*5. - self.V3) / (2 * self.V4)))
        W_inf = (1 / 2) * (1 + bm.tanh((V*5. - self.V3) / self.V4))
        dWdt = (W_inf - W) / tau_W
        return dWdt

    self.int_V = bp.odeint(dV, method=method)
    self.int_W = bp.odeint(dW, method=method)

  def update(self, tdi):
    V = self.int_V(self.V, tdi.t, self.W, self.input, tdi.dt)
    self.W.value = self.int_W(self.W, tdi.t, self.V, tdi.dt)
    self.spike.value = bm.logical_and(self.V < self.V_th, V >= self.V_th)
    self.V.value = V
    self.input[:] = 0.

In [3]:
s =bm.array([0.5,1,1.5])
phi = bm.array([[0.3313,0.8148,0.4364],[0.8835,0.3621,0.2182],[0.3313,0.4527,0.8729]])
b = bm.dot(phi.T,s)
w = bm.dot(phi.T,phi)
w[bm.diag_indices_from(w)] = 0
print(w)
print(b)

Array(value=Array([[0.        , 0.7397555 , 0.62646115],
                   [0.7397555 , 0.        , 0.82969487],
                   [0.62646115, 0.82969487, 0.        ]]),
      dtype=float32)
Array(value=Array([1.5460999, 1.44855  , 1.74575  ]), dtype=float32)


In [19]:
class Spiking_LCA(bp.DynamicalSystemNS):
    def __init__(self, num, w, b, lamb, method='exp_auto'):
        super(Spiking_LCA, self).__init__()
        
        # parameter setting
        self.lamb = lamb
        
        # network size
        num_neuron = int(num)
        self.N = ML_Scale(num_neuron)
        self.average_current = bm.Variable(bm.zeros(num_neuron))          # 平均电流记录
        # synapses
        self.N2N = bp.synapses.Exponential(pre=self.N,
                                            post=self.N,
                                            conn= bp.connect.All2All(),
                                            g_max= w,
                                            tau=1.,
                                            output=bp.synouts.CUBA(),
                                            method=method)

    # def reversed_function(self,x):
    #     return 7.43164314*x**4 -14.61153623* x**3 + 13.71690213 *x**2 -3.86053151*x + 4.32149502

    def reversed_function(self, x):
        return 8.35304086 * x**5 - 22.99371634 * x**4 + 26.47121174 * x**3 - 11.32892117 * x**2 + 2.74156822 * x + 3.76677658


    def update(self):
        t = bp.share.load('t')
        dt = bp.share.load('dt')
        self.N2N()
        self.average_current.value = self.average_current *t/(t + 1) + (b - self.N2N.g)/(t+1)
        firing_rate  =  bm.maximum(self.average_current - self.lamb, 0.)
        self.N.input =  bm.where(firing_rate > 1e-7, self.reversed_function(firing_rate) , 3.9)
        self.N()

In [20]:
net    = Spiking_LCA(w.shape[0],w,b,0.1)
total_period = 3000
runner = bp.DSRunner(net,monitors=['N.V','N.spike'], dt = 0.01)
runner.run(total_period)

Predict 300000 steps: :   0%|          | 0/300000 [00:00<?, ?it/s]

Predict 300000 steps: : 100%|██████████| 300000/300000 [01:47<00:00, 2791.13it/s] 


In [21]:
size_num = runner.mon['N.spike'].shape[0]
print(size_num)
spike_calculate = runner.mon['N.spike'][size_num*3//4:,:]
sum_along_columns = np.sum(spike_calculate, axis=0)/total_period*4
print(sum_along_columns)

300000
[0.68       0.         1.21866667]
